# Data Clean

In [ ]:
import pandas as pd
from pyreadr import read_r
import numpy as np

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [ ]:
# Load the data
ling_data = pd.read_csv('../data/lingData.txt', sep='\\s+')
ling_location = pd.read_csv('../data/lingLocation.txt', sep='\\s+')

# ling_data has a column for each question, and ling_location has a column
# for each question x answer.  Sorry the columns in ling_location are not usefully named,
# but it's not too tricky to figure out which is which.
# Note that you still need to clean this data (check for NA's, missing location data, etc.)

# Load the question_data which contains quest.mat, quest.use, ans.---
question_data = read_r('../data/question_data.RData')

In [ ]:
# check shape of ling_data and ling_location
print(ling_data.shape) # 47471 * 73
print(ling_location.shape) # 781 * 471

(47471, 73)
(781, 471)


In [ ]:
# checking invalid data or missing entries of ling_data
pd.set_option('display.max_columns', None)
a = ling_data.select_dtypes(include=[np.number, 'bool']).describe()
print(a)
# count of lat and long are all 46451 (missing entries)

                 ID           ZIP          Q050          Q051          Q052  \
count  47471.000000  47471.000000  47471.000000  47471.000000  47471.000000   
mean   25427.002823  48664.195024      5.177119      1.675612      1.900718   
std    14394.594430  29477.582590      2.645842      0.565226      0.832977   
min        1.000000   1001.000000      0.000000      0.000000      0.000000   
25%    13115.500000  20770.000000      4.000000      1.000000      1.000000   
50%    25555.000000  49015.000000      4.000000      2.000000      2.000000   
75%    37936.500000  70806.000000      7.000000      2.000000      3.000000   
max    50064.000000  99901.000000      9.000000      3.000000      3.000000   

               Q053          Q054          Q055          Q056          Q057  \
count  47471.000000  47471.000000  47471.000000  47471.000000  47471.000000   
mean       1.839818      1.909608      1.909461      1.636220      1.789008   
std        0.452784      0.432830      0.464743    

In [ ]:
# checking invalid data or missing entries of ling_data
# look at missing entries:
missing = ling_data.isna().sum()
missing_sort = missing.sort_values(ascending=False)
print(missing_sort.head(5))
# there are 1020 missing vals in long and lat, 540 missing vals in city and 3 missing vals in state

long     1020
lat      1020
CITY      540
STATE       3
Q092        0
dtype: int64


In [ ]:
# checking ling_location
print(ling_location.head(10))

    Number of people in cell  Latitude  Longitude  V4  V5  V6  V7  V8  V9  \
1                          2        19       -155   0   0   0   2   0   0   
2                          4        20       -155   0   0   0   2   0   0   
3                          3        20       -156   0   0   0   3   0   0   
4                          7        21       -156   1   0   0   2   0   0   
5                          1        21       -157   0   0   0   0   0   0   
6                         84        21       -158   9   0   0  50   0   0   
7                          9        22       -158   0   0   0   6   0   0   
8                          3        25        -80   0   0   0   0   0   0   
9                          4        25        -81   0   1   0   1   0   0   
10                         1        25        -82   0   0   0   0   0   0   

    V10  V11  V12  V13  V14  V15  V16  V17  V18  V19  V20  V21  V22  V23  V24  \
1     0    0    0    1    0    1    1    1    0    0    2    0    0    

In [ ]:
# calculate total # of people in ling_location
column_sum = ling_location.iloc[:, 0].sum()  # iloc[:, 0] selects the first column
column_sum
# why ling_data have missing values in long and lat, but here sum is still 47471?"

47471

In [ ]:
# we need to round lat and long since in ling_location both are all integers

# Create a copy of the original DataFrame to avoid modifying it
rounded_lingdata = ling_data.copy()

# Round the 'long' and 'lat' columns to the nearest integer using the 'round' method and assign the results back to the corresponding columns in the new DataFrame
rounded_lingdata['long'] = rounded_lingdata['long'].round()
rounded_lingdata['lat'] = rounded_lingdata['lat'].round()

# count unique values of long and lat in ling_data separately
rounded_lingdata['long'].value_counts()
rounded_lingdata['lat'].value_counts()

lat
41.0    6427
42.0    6354
39.0    4988
40.0    4904
43.0    3463
34.0    2291
38.0    2129
30.0    1837
45.0    1721
37.0    1671
33.0    1518
48.0    1442
36.0    1310
44.0    1193
35.0    1171
47.0     837
46.0     800
32.0     549
26.0     419
31.0     394
28.0     319
29.0     265
27.0     136
49.0      99
21.0      92
61.0      46
65.0      19
22.0       9
25.0       8
60.0       7
20.0       7
57.0       6
58.0       6
62.0       5
55.0       4
19.0       2
71.0       2
64.0       1
Name: count, dtype: int64

In [ ]:
# Filter the DataFrame by the specified latitude value
filtered_df = ling_location[ling_location['Latitude'] == 41]

# Calculate the sum of the 'Number of people in cell' column from the filtered DataFrame
sum_people = filtered_df.iloc[:, 0].sum()  # Assuming the first column is 'Number of people in cell
sum_people
# sum_people (latitude is 41) is exactly the same as the long_data after rounding"

6427

In [ ]:
# follow the same procedure, we can check whether the sum of the 'Number of people in cell' of other latitude value is the same

# Extract unique 'lat' values
unique_latitudes = ling_location['Latitude'].unique()

# Initialize a dictionary to store the sums for each unique 'lat' value
sum_people_by_latitude = {}

# Iterate over unique 'lat' values
for latitude in unique_latitudes:
    # Filter the DataFrame by the current 'lat' value\n
    filtered_df = ling_location[ling_location['Latitude'] == latitude]

    # Calculate the sum of the 'Number of people in cell' column from the filtered DataFrame
    sum_people = filtered_df.iloc[:, 0].sum()  # Assuming the first column is 'Number of people in cell'

    # Store the sum in the dictionary with the 'lat' value as the key
    sum_people_by_latitude[latitude] = sum_people

    # Sort the dictionary by values in descending order
    sorted_sum_people_by_latitude = dict(sorted(sum_people_by_latitude.items(), key=lambda item: item[1], reverse=True))

# Print the result
print(sorted_sum_people_by_latitude)
# We found that in the ling_location, they made lat of (1020) NA in ling_data as 90

{41: 6427, 42: 6354, 39: 4988, 40: 4904, 43: 3463, 34: 2291, 38: 2129, 30: 1837, 45: 1721, 37: 1671, 33: 1518, 48: 1442, 36: 1310, 44: 1193, 35: 1171, 90: 1020, 47: 837, 46: 800, 32: 549, 26: 419, 31: 394, 28: 319, 29: 265, 27: 136, 49: 99, 21: 92, 61: 46, 65: 19, 22: 9, 25: 8, 20: 7, 60: 7, 57: 6, 58: 6, 62: 5, 55: 4, 19: 2, 71: 2, 64: 1}


In [ ]:
# follow the same procedure above, we can check whether the sum of the 'Number of people in cell' of other longitude value is the same

# Extract unique 'long' values
unique_longitudes = ling_location['Longitude'].unique()

# Initialize a dictionary to store the sums for each unique 'lat' value
sum_people_by_longitude = {}

# Iterate over unique 'lat' values
for longitude in unique_longitudes:
    # Filter the DataFrame by the current 'lat' value
    filtered_df = ling_location[ling_location['Longitude'] == longitude]

    # Calculate the sum of the 'Number of people in cell' column from the filtered DataFrame
    sum_people = filtered_df.iloc[:, 0].sum()  # Assuming the first column is 'Number of people in cell'

    # Store the sum in the dictionary with the 'lat' value as the key
    sum_people_by_longitude[longitude] = sum_people

    # Sort the dictionary by values in descending order\n",
    sorted_sum_people_by_longitude = dict(sorted(sum_people_by_longitude.items(), key=lambda item: item[1], reverse=True))

# Print the result
print(sorted_sum_people_by_longitude)
# We found that in the ling_location, they made long of (1020) NA in ling_data as -170

{-74: 3465, -88: 2823, -122: 2590, -95: 2141, -77: 1810, -83: 1787, -84: 1678, -75: 1645, -73: 1427, -71: 1411, -76: 1403, -97: 1402, -86: 1363, -80: 1341, -93: 1300, -90: 1232, -94: 1136, -82: 1090, -118: 1065, -92: 1048, -170: 1020, -96: 990, -87: 899, -123: 880, -81: 849, -98: 834, -85: 832, -89: 771, -79: 700, -117: 688, -91: 645, -112: 618, -105: 605, -72: 538, -78: 489, -119: 323, -121: 291, -99: 229, -120: 215, -111: 171, -107: 167, -70: 138, -109: 132, -106: 121, -102: 114, -101: 107, -108: 101, -116: 100, -158: 93, -124: 81, -115: 79, -114: 68, -100: 66, -103: 64, -104: 54, -69: 47, -150: 43, -110: 32, -113: 29, -68: 13, -148: 13, -156: 10, -67: 8, -135: 8, -151: 8, -155: 6, -147: 6, -149: 5, -132: 4, -157: 3, -133: 2, -153: 2, -145: 2, -161: 1}


In [ ]:
# So here we will not directly delete the NA (long and lat) rows in rounded_lingdata, we will use long (-170) and lat (90) to impute these NAs.
# Why we do this? Since we can observe missing # of city and state are 540 and 3 respectively, which means we can get long and lat from non-missing # of cities and states; Maybe we can check # of missing city and states later.
impute_long = -170
impute_lat = 90

# Impute the missing values in the 'long' column with the specified longitude
rounded_lingdata['long'] = rounded_lingdata['long'].fillna(impute_long)

# Impute the missing values in the 'lat' column with the specified latitude
rounded_lingdata['lat'] = rounded_lingdata['lat'].fillna(impute_lat)

# check missing entries again
missingr = rounded_lingdata.isna().sum()
missingr_sort = missingr.sort_values(ascending=False)
print(missingr_sort.head(5))

CITY     540
STATE      3
Q092       0
Q099       0
Q098       0
dtype: int64


In [ ]:
# look at missing entries of city and state, and check their corresponding lat and long
# Filter the DataFrame for rows where 'city' and 'state' are both NA
na_rows = rounded_lingdata[rounded_lingdata['CITY'].isna() & rounded_lingdata['STATE'].isna()]

# Print the filtered entries
print(na_rows)
# it's an empty data frame, which means each entry must have at least one of the 'state' or 'city', from which we can imply both long and lat. So we are not going to delete any entries in the rounded_lingdata (or ling_data)

Empty DataFrame
Columns: [ID, CITY, STATE, ZIP, Q050, Q051, Q052, Q053, Q054, Q055, Q056, Q057, Q058, Q059, Q060, Q061, Q062, Q063, Q064, Q065, Q066, Q067, Q068, Q069, Q070, Q071, Q072, Q073, Q074, Q075, Q076, Q077, Q078, Q079, Q080, Q081, Q082, Q083, Q084, Q085, Q086, Q087, Q088, Q089, Q090, Q091, Q092, Q093, Q094, Q095, Q096, Q097, Q098, Q099, Q100, Q101, Q102, Q103, Q104, Q105, Q106, Q107, Q109, Q110, Q111, Q115, Q117, Q118, Q119, Q120, Q121, lat, long]
Index: []


In [ ]:
# impute city and state
na_rows = rounded_lingdata[rounded_lingdata['CITY'].isna() | rounded_lingdata['STATE'].isna()]

# Create a copy of the original DataFrame to avoid modifying it
name_ling = rounded_lingdata.copy()

# Group the DataFrame by 'long' and 'lat', and then transform to fill 'city' where it is NA
name_ling['CITY'] = name_ling.groupby(['long', 'lat'])['CITY'].transform(lambda x: x.bfill().ffill())
name_ling['STATE'] = name_ling.groupby(['long', 'lat'])['STATE'].transform(lambda x: x.bfill().ffill())

# check missing again
missing = name_ling.isna().sum()
missing_sort = missing.sort_values(ascending=False)
print(missing_sort.head(5))

ID      0
Q083    0
Q099    0
Q098    0
Q097    0
dtype: int64


In [ ]:
# checking # of choices in each question in order to check ling_location (781, 471) V4-V471 vector (check dimension of ling_location)

# Assuming question_data is the dictionary with various keys including 'ans.##' patterns

# Initialize an empty dictionary to store the count of different answers for each question
answer_counts = {}

# Iterate over each key in the dictionary keys
for q_key in question_data.keys():
    # Check if the key starts with 'ans.' and has a numeric portion
    if q_key.startswith('ans.') and q_key.count('.') == 1:
        try:
             # Extract the question number from the key (e.g., 'ans.120' -> 120)
             q_num = int(q_key.split('.')[1])

             # Ensure the question number falls within your desired range
             if 50 <= q_num <= 121:
                 # Count the number of unique answers for this question
                 # Since each row represents a unique answer, the count of rows is the number of unique answers
                 num_answers = len(question_data[q_key])

                 # Store the count in the answer_counts dictionary using the question number as the key\n",
                 answer_counts[q_num] = num_answers

        except ValueError:
             # Handle the exception if conversion fails
             print(f"Skipping key {q_key}: cannot convert to integer.")

    # Print the answer counts for each question
    for q_num, count in answer_counts.items():
       print(f"Question {q_num} has {count} different answers.")

Question 100 has 5 different answers.
Question 100 has 5 different answers.
Question 101 has 5 different answers.
Question 100 has 5 different answers.
Question 101 has 5 different answers.
Question 102 has 11 different answers.
Question 100 has 5 different answers.
Question 101 has 5 different answers.
Question 102 has 11 different answers.
Question 103 has 5 different answers.
Question 100 has 5 different answers.
Question 101 has 5 different answers.
Question 102 has 11 different answers.
Question 103 has 5 different answers.
Question 104 has 6 different answers.
Question 100 has 5 different answers.
Question 101 has 5 different answers.
Question 102 has 11 different answers.
Question 103 has 5 different answers.
Question 104 has 6 different answers.
Question 105 has 10 different answers.
Question 100 has 5 different answers.
Question 101 has 5 different answers.
Question 102 has 11 different answers.
Question 103 has 5 different answers.
Question 104 has 6 different answers.
Questi

In [ ]:
# Calculate the sum of all values in the answer_counts dictionary
total_answers = sum(answer_counts.values())

# Print the total sum of answers
print("The total number of answers is:", total_answers)

The total number of answers is: 485


In [ ]:
# List the column names for which you want to calculate the maximum values
column_names = ['Q050', 'Q051', 'Q052', 'Q053', 'Q054',
                'Q055', 'Q056', 'Q057', 'Q058', 'Q059', 'Q060', 'Q061', 'Q062', 'Q063',
                'Q064', 'Q065', 'Q066', 'Q067', 'Q068', 'Q069', 'Q070', 'Q071', 'Q072',
                'Q073', 'Q074', 'Q075', 'Q076', 'Q077', 'Q078', 'Q079', 'Q080', 'Q081',
                'Q082', 'Q083', 'Q084', 'Q085', 'Q086', 'Q087', 'Q088', 'Q089', 'Q090',
                'Q091', 'Q092', 'Q093', 'Q094', 'Q095', 'Q096', 'Q097', 'Q098', 'Q099',
                'Q100', 'Q101', 'Q102', 'Q103', 'Q104', 'Q105', 'Q106', 'Q107', 'Q109',
                'Q110', 'Q111', 'Q115', 'Q117', 'Q118', 'Q119', 'Q120', 'Q121']

# Calculate the maximum values for the specified columns
max_values = rounded_lingdata[column_names].max()

# Calculate the sum of the maximum values
sum_of_max_values = max_values.sum()

# Print the sum of the maximum values
print("The sum of the maximum values is:", sum_of_max_values)
# match the 471 of ling_location, but why different from # of questions in question data? Some questions are skipped.

The sum of the maximum values is: 468


In [ ]:
# check missing in ling_location
# check missing entries again
missingloc = ling_location.isna().sum()
missingloc_sort = missingloc.sort_values(ascending=False)
print(missingloc_sort.head(5))
# no missing entries

Number of people in cell    0
V311                        0
V323                        0
V322                        0
V321                        0
dtype: int64


In [ ]:
# normalize data in ling_location
lingloc_normalize = ling_location.copy()
lingloc_normalize.iloc[:, 3:471] = lingloc_normalize.iloc[:, 3:471].div(lingloc_normalize.iloc[:, 0], axis=0)

/var/folders/2_/8561wcj10390hzz8d8ydnk980000gn/T/ipykernel_50787/4053628917.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1      0.000000
2      0.000000
3      0.000000
4      0.142857
5      0.000000
         ...   
777    0.000000
778    0.000000
779    0.076923
780    0.500000
781    0.140196
Name: V4, Length: 781, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lingloc_normalize.iloc[:, 3:471] = lingloc_normalize.iloc[:, 3:471].div(lingloc_normalize.iloc[:, 0], axis=0)
/var/folders/2_/8561wcj10390hzz8d8ydnk980000gn/T/ipykernel_50787/4053628917.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1      0.000000
2      0.000000
3      0.000000
4      0.000000
5      0.000000
         ...   
777    0.000000
778    0.000000
779    0.000000
780    0.000000
781    0.006863
Name: V5, 

/var/folders/2_/8561wcj10390hzz8d8ydnk980000gn/T/ipykernel_50787/4053628917.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1      0.000000
2      0.000000
3      0.000000
4      0.000000
5      0.000000
         ...   
777    0.000000
778    0.000000
779    0.000000
780    0.000000
781    0.013725
Name: V197, Length: 781, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lingloc_normalize.iloc[:, 3:471] = lingloc_normalize.iloc[:, 3:471].div(lingloc_normalize.iloc[:, 0], axis=0)
/var/folders/2_/8561wcj10390hzz8d8ydnk980000gn/T/ipykernel_50787/4053628917.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1      0.000000
2      0.250000
3      0.000000
4      0.000000
5      0.000000
         ...   
777    0.000000
778    0.000000
779    0.000000
780    0.000000
781    0.040196
Name: V1

In [ ]:
clean_ling_data = name_ling
clean_lingloc = lingloc_normalize

In [ ]:
# save as csv
# clean_ling_data.to_csv('../data/ling_data.csv', index=False)
# clean_lingloc.to_csv('../data/clean_lingloc.csv', index=False)